# Data Cleaning

## Library Imports

In [1]:
import pandas as pd

## Data Imports

In [2]:
# read vaccination rate csv
vaccination = pd.read_csv('../datasets/vaccination_rate.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../datasets/vaccination_rate.csv'

In [ ]:
# read vaccine hesitancy csv
hesitancy = pd.read_csv('../datasets/vaccine_hesitancy.csv')

### Vaccination Rate Dataset

In [ ]:
# look at top five rows
vaccination.head()

In [ ]:
# change column names to lowercase
vaccination.columns = vaccination.columns.str.lower()

In [ ]:
vaccination.info()

In [ ]:
vaccination.describe()

In [ ]:
# drop duplicates
# keeps the most recent vaccination rate
vaccination = vaccination.drop_duplicates(subset=['fips'])

In [ ]:
vaccination.info()

- `fips` should not be object

In [ ]:
# remove row with unknown county
delete_row = vaccination[vaccination['fips'] == 'UNK'].index
vaccination = vaccination.drop(delete_row)

In [ ]:
# convert fips column to float
vaccination['fips_code']= (vaccination['fips'].astype(int)/1)

In [ ]:
vaccination.isnull().sum()

In [ ]:
#locate which row has null in svi_ctgy 
vaccination[vaccination['svi_ctgy'].isna()]

In [ ]:
# remove rows
vaccination = vaccination.drop([109, 429, 667, 3041])

In [ ]:
# fill nulls with '0'
vaccination.fillna('0', inplace=True)

In [ ]:
# svi_ctgy for Rio Arriba County with 'D'
vaccination['svi_ctgy'].replace({ '0':'D'}, inplace=True)

In [ ]:
# drop columns
vaccination = vaccination.drop(columns=['date', 'mmwr_week', 'fips'])

In [ ]:
# save cleaned data
vaccination.to_csv('../cleaned_data/vac_rate.csv', index = False)

### Vaccine Hesitancy Dataset

In [ ]:
# look at top five rows
hesitancy.head()

In [ ]:
# change column names to lowercase
# replace '' with '_'
hesitancy.columns = hesitancy.columns.str.replace(' ','_').str.lower()

In [ ]:
hesitancy.info()

In [ ]:
hesitancy.describe()

In [ ]:
hesitancy.isnull().sum()

In [ ]:
# fill nulls with '0'
hesitancy['percent_adults_fully_vaccinated_against_covid-19_(as_of_6/10/21)'].fillna(0, inplace=True)

In [ ]:
# create new column with just the county name
hesitancy['name_of_county'] = hesitancy['county_name'].str.split(',').str[0]

In [ ]:
#locate which row has null 
hesitancy[hesitancy['social_vulnerability_index_(svi)'].isna()]

In [ ]:
hesitancy.groupby(['state']).mean()

In [ ]:
# using the mean svi of New Mexico to replace the null value of Rio Arriba County, New Mexico with category
hesitancy['social_vulnerability_index_(svi)'].fillna('.79', inplace=True)
hesitancy['svi_category'].fillna('.', inplace=True)

In [ ]:
# drop columns
hesitancy = hesitancy.drop(columns=['county_name', 'geographical_point', 'county_boundary', 'state_boundary'])

In [ ]:
# save cleaned data
hesitancy.to_csv('../cleaned_data/vaccine_hest.csv', index = False)

### Merge Datasets

In [ ]:
# merge vaccination rate with vaccine hesitancy 
merged = pd.merge(hesitancy, vaccination, on = 'fips_code')

In [ ]:
# drop columns with same information 
merged.drop(columns=['state_code', 'name_of_county'], inplace=True)

In [ ]:
# drop rows with  vaccination rate of 0 
merged.drop(merged[merged['series_complete_pop_pct'] == 0].index, inplace = True)

In [ ]:
# save cleaned data
merged.to_csv('../cleaned_data/merged.csv', index = False)

### Add Vaccine Allocation

In [ ]:
state = pd.read_csv('../datasets/state_allocation.csv')
merged = pd.read_csv('../cleaned_data/merged.csv')

In [ ]:
state = state[state['Date'] == '12/21/2021']

#looking only the date that corresponds with our other data

In [ ]:
merged['pop'] = round(merged['series_complete_yes'] / (merged['series_complete_pop_pct'] / 100))

In [ ]:
state_pop = {'AL':4934193, 'AK':724357, 'AZ':7520103, 'AR':3033946, 'CA':39613493, 'CO':5893634, 'CT':3552821, 'DE':990334, 'DC':714153, 
             'FL':21944577, 'GA':10830007, 'HI':1406430, 'ID':1860123, 'IL':12569321, 'IN':6805663, 'IA':3167974, 'KS':2917224, 'KY':4480713, 
             'LA':4627002, 'ME':1354522, 'MD':6065436, 'MA':6912239, 'MI':9992427, 'MN':5706398, 'MS':2966407, 'MO':6169038, 'MT':1085004, 
             'NE':1951996, 'NV':3185786, 'NH':1372, 'NJ':8874520, 'NM':2105005, 'NY':19299981, 'NC':10701022, 'ND':770026, 'OH':11714618, 
             'OK':3990443, 'OR':4289439, 'PA':12804123, 'PR':3194374, 'RI':1061509, 'SC':5277830, 'SD':896581, 'TN':6944260, 'TX':29730311, 
             'UT':3310774, 'VT':623251, 'VA':8603985, 'WA':7796941, 'WV':1767859, 'WI':5852490, 'WY':581075}

In [ ]:
merged['state_pop'] = merged['recip_state']
merged['state_pop'] = merged['state_pop'].map(state_pop)

In [ ]:
merged['state_pop_pct'] = (merged['pop'] / merged['state_pop']) * 100

In [ ]:
state_dist = dict(zip(state.Location, state.Distributed))

In [ ]:
merged['distributed'] = merged['recip_state']
merged['distributed'] = merged['distributed'].map(state_dist)

In [ ]:
merged['distributed'] = round((merged['distributed'] * merged['state_pop_pct']) / 100)

In [ ]:
merged.head()

In [ ]:
merged.to_csv('../cleaned_data/merged.csv', index = False)